In [12]:
import geopandas as gpd
import pandas as pd

In [4]:
buildings = gpd.read_file('buildings.geojson')

In [5]:
reports = gpd.read_file('reports.geojson')

In [6]:
messages = gpd.read_file('messages.geojson')

In [4]:
buildings.head(3)

,index,building_id,address,project_type,building_year,repair_years,building_area,living_area,storeys_count,central_heating,...,failure,is_living,resident_number,population_balanced,city,administrative_unit,municipality,block_id,kindergartens_provision_value,geometry
0,0,137539,"Санкт-Петербург, посёлок Парголово, садоводчес...",NaN,NaN,NaN,93.9880,NaN,NaN,0,...,0,True,NaN,0,Санкт-Петербург,Выборгский район,Парголово,164496.0,NaN,"POLYGON ((30.18608 60.11278, 30.18625 60.11275..."
1,1,137585,"Санкт-Петербург, посёлок Парголово",NaN,NaN,NaN,37.3141,NaN,NaN,0,...,NaN,False,NaN,0,Санкт-Петербург,Выборгский район,Парголово,164616.0,NaN,"POLYGON ((30.23020 60.07636, 30.23033 60.07637..."
2,2,137603,"Санкт-Петербург, Успенка, 39а",NaN,NaN,NaN,70.3008,49.2105,1.0,0,...,0,True,NaN,1,Санкт-Петербург,Выборгский район,Парголово,164616.0,NaN,"POLYGON ((30.22085 60.07275, 30.22079 60.07279..."


In [5]:
buildings.columns

Index(['index', 'building_id', 'address', 'project_type', 'building_year',
       'repair_years', 'building_area', 'living_area', 'storeys_count',
       'central_heating', 'central_hotwater', 'central_water',
       'central_electro', 'central_gas', 'refusechute', 'ukname', 'lift_count',
       'failure', 'is_living', 'resident_number', 'population_balanced',
       'city', 'administrative_unit', 'municipality', 'block_id',
       'kindergartens_provision_value', 'geometry'],
      dtype='object')

In [7]:
services = gpd.read_file('services.geojson')

In [21]:
services.head(3)

,id,name,website,capacity,municipality,address,opening_hours,city_service_type,administrative_unit,block_id,phone,geometry
0,155966.0,Народная аптека,NaN,292.0,Ланское,"Санкт-Петербург, Ланское шоссе, 24 к1",24/7,Аптека,Приморский,944.0,NaN,POINT (30.32163 59.99656)
1,155829.0,Аптека на Боровой,NaN,706.0,Владимирский округ,NaN,Mo-Fr 09:00-21:00; Sa-Su 11:00-19:00,Аптека,Центральный,330.0,NaN,POINT (30.34591 59.91939)
2,155830.0,Первая Помощь,https://superapteka.ru/,15.0,Балканский,"Санкт-Петербург, Будапештская, 83, Дунайский п...",Mo-Su 09:00-21:00,Аптека,Фрунзенский,587.0,NaN,POINT (30.39644 59.83794)


In [23]:
services.city_service_type.unique()

array(['Аптека', 'Больница', 'Детский сад', 'Музей',
       'Объект культурного наследия', 'Универсам/супермаркет/гипермаркет',
       'Школа'], dtype=object)

In [8]:
print("Q: На что мне обратить внимание по вопросам стратегии развития города?")
MEAN_KINDERGARTEN_PROVISION = int(buildings.kindergartens_provision_value.dropna().mean().round(2) * 100)
TARGET_KINDERGARTEN_PROVISION = 100
print(f"A: В сфере образования наблюдается разрыв между плановым показателем в {TARGET_KINDERGARTEN_PROVISION}% обеспеченности местами в детских садах и текущим показателем в {MEAN_KINDERGARTEN_PROVISION}%")

Q: На что мне обратить внимание по вопросам стратегии развития города?
A: В сфере образования наблюдается разрыв между плановым показателем в 100% обеспеченности местами в детских садах и текущим показателем в 35%


In [26]:
print("Q: Какой на данный момент дефицит мест в детских садах в Московском районе?")
kindergartens = services[services['city_service_type'] == 'Детский сад']
kindergartens = kindergartens[kindergartens['administrative_unit'] == 'Московский']
NUMBER_OF_PLACES_IN_KINDERGARTENS = int(kindergartens.capacity.sum()) #fix with true value
print(f"A: В Московском районе не хватает {NUMBER_OF_PLACES_IN_KINDERGARTENS} мест в детских садах")


Q: Какой на данный момент дефицит мест в детских садах в Московском районе?
A: В Московском районе не хватает 22387 мест в детских садах


In [9]:
print(f"Q: В каком районе наиболее активно жалуются на благоустройство?")
problem_df = reports[reports['subcategory'] == 'Благоустройство']
district_counts = problem_df.groupby('district').size()
max_complaint_district = district_counts.idxmax()
print(f"A: {max_complaint_district} район имеет наибольшее количеством жалоб на благоустройство")

Q: В каком районе наиболее активно жалуются на благоустройство?
A: Невский район имеет наибольшее количеством жалоб на благоустройство


In [10]:
print(f'Q: В каком муниципалитете складыается проблемная ситуация и проблемы в каких сферах в нём присутсвуют?')
municipality_problems = reports.groupby('municipality')['subcategory'].count()
max_problem_municipality = municipality_problems.idxmax()
top_problems = reports[reports['municipality'] == max_problem_municipality]['subcategory'].value_counts().nlargest(5)
top_problems_str = top_problems.to_string()
top_problems_str = '\n'.join([line for line in top_problems_str.split('\n') if not line.strip().endswith('int64')])
print(f"A: Муниципалитет с наибольшим количеством проблем: {max_problem_municipality}. Главными проблемами, которые упоминают жители являются:\n{top_problems_str}")

Q: В каком муниципалитете складыается проблемная ситуация и проблемы в каких сферах в нём присутсвуют?
A: Муниципалитет с наибольшим количеством проблем: муниципальный округ Ульянка. Главными проблемами, которые упоминают жители являются:
Содержание МКД                                             6399
Благоустройство                                            3947
Незаконная информационная и (или) рекламная конструкция     169
Санитарное состояние                                        118
Фасад                                                       109


In [13]:
print("Q: Какие проблемы в городе не решены в течение месяца?")
messages['date'] = pd.to_datetime(messages['date'])
messages = messages.sort_values(by='date')
grouped = messages.groupby('function')
result = grouped.agg( count=('function', 'size'), first_date=('date', 'first'), last_date=('date', 'last')).reset_index()
result = (result.sort_values(by='count', ascending=False) .reset_index(drop=True) .rename(columns={ 'function': 'Область', 'count': 'Количество упоминаний', 'first_date': 'Первое упоминание', 'last_date': 'Последнее упоминание'}))
print("A: В течение месяца не решены проблемы в следующих областях:")
result.head(5)

Q: Какие проблемы в городе не решены в течение месяца?
A: В течение месяца не решены проблемы в следующих областях:


,Область,Количество упоминаний,Первое упоминание,Последнее упоминание
0,Другое,3262,2024-01-01,2024-01-19
1,Безопасность,1089,2024-01-01,2024-01-19
2,ЖКХ,880,2024-01-01,2024-01-19
3,Благоустройство,747,2024-01-01,2024-01-19
4,Энергетика,678,2024-01-01,2024-01-19
